# **OJKGPT**

This is the Jupyter Notebook for the development of the 1st version of OJKGPT for the showcase demo capabilites to wealth team program. <br>
In this development, we use several data sources for enriching the RAG system of the chatbot. 

In [13]:
import nest_asyncio
nest_asyncio.apply()

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [16]:
from utils.models_define import ModelName

# storing or not
STORE = False
DELETE = False
model_name = ModelName.OPENAI

## **LLM Models Define**

In [17]:
from llama_index.core import Settings
from utils.models_define import get_llm_and_embedding

# ollama/openai/azure_openai
llm, embedding_llm = get_llm_and_embedding(model_name=model_name)

Settings.llm = llm
Settings.embed_model = embedding_llm

## **Loading Documents**

### **Document Reader and Node Parser**

In [18]:
from utils.documents_reader import read_documents
from utils.node_parser import parse_nodes
from utils.doc_store import store_docstore, load_docstore

path = './data'

if STORE:
    docs_all = read_documents(path=path)
    nodes_all = parse_nodes(documents=docs_all, llm=llm)
    docstore = store_docstore(documents=docs_all, delete=DELETE)
else:
    docstore = load_docstore()

## **Indexing & Storing**

In [19]:
from utils.vector_store import (store_vector_index, load_vector_index)

# store
if STORE:
    index_all = store_vector_index(nodes=nodes_all,embed_model=embedding_llm, delete=DELETE)

# load
else:
    index_all = load_vector_index()

Loading the index completed.


## **Querying**

### **Retriever**

In [20]:
from llama_index.retrievers.bm25 import BM25Retriever
from utils.fusion_retriever import FusionRetriever
# from llama_index.core.retrievers import RecursiveRetriever

vector_retriever_all = index_all.as_retriever(similarity_top_k=5)

bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore, similarity_top_k=5, verbose=True
)

fusion_retriever = FusionRetriever(
    llm=llm,
    retrievers=[vector_retriever_all, bm25_retriever],
    similarity_top_k=5,
)

### **Query Engine**

In [21]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine_all = RetrieverQueryEngine.from_args(retriever=fusion_retriever,llm=llm)

In [120]:
a = "Apa judul peraturan 7/33/PBI/2005?" # Pencabutan atas Peraturan Bank Indonesia Nomor 5/17/PBI/2003 tentang Persyaratan dan Tata Cara Pelaksanaan Jaminan Pemerintah terhadap Kewajiban Pembayaran Bank Perkreditan Rakyat
b = "Kapan surat edaran No. 15/26/DPbS mulai berlaku?" # 1 Agustus 2013.
c = "Siapa nama dan jabatannya yang menandatangani surat dengan nomor 1/SEOJK.04/2013?" # NURHAIDA, kepala eksekutif pengawas pasar modal
d = "Saya ingin menyelenggarakan kegiatan pasar modal berikan saya nomor surat yang membahas mengenai hal ini!" # 1/SEOJK.04/2013
e = "Berapa persen jaminan moneter pada tanggal 20 Agustus 1958?" # 7,3%
f = "Surat edaran nomor berapa yang mengatur bank umum syariah dan unit usaha syariah?" # 15/26/DPbS
g = "Apa kepanjangan dari PAPSI?" # Pedoman Akuntansi Perbankan Syariah Indonesia
h = "apa judul peraturan nomor 112/KMK.03/2001?" # Keputusan Menteri Keuangan tentang Pemotongan Pajak Penghasil Pasal 21 atas Penghasilan berupa Uang Pesangon, Uang Tebusan Pensiun, dan Tunjangan Hari Tua atau Jaminan Hari Tua
i = "Saya ingin membuat sistem informasi lembaga jasa keuangan, berikan nomor regulasi dari peraturan yang membahas tentang manejemen risiko nya!" # 4/POJK.05/2021
j = "Apa kepanjangan dari SWDKLLJ?" # Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan
k = "Berapa nilai SWDKLLJ Jeep?" # Rp.140.000
l = "Apa latar belakang dari peraturan NOMOR 4/POJK.05/2021?" # dalam bentuk list
m = "Apa itu LJKNB?" # Lembaga Jasa Keuangan Non Bank
n = "Apakah KMK Nomor 462/KMK.04/1998 masih berlaku" # tidak
o = "Apa itu Uang Pesangon?" # penghasilan yang dibayarkan oleh pemberi kerja kepada karyawan dengan nama dan dalam bentuk apapun sehubungan dengan berakhirnya masa kerja atau terjadi pemutusan  hubungan kerja, termasuk uang penghargaan masa kerja dan uang  ganti kerugian
p = "Apa itu CKPN?" # Cadangan Kerugian Penurunan Nilai.

z = "What is the limit of QRIS Transaction Nominal?"

query_str = k

In [121]:
response = query_engine_all.query(query_str)

100%|██████████| 6/6 [00:00<00:00,  6.96it/s]


In [122]:
from llama_index.core.response.notebook_utils import display_response

display_response(response)

**`Final Response:`** Rp140.000,00

## **Build Agent**

In [123]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_tools = [
    QueryEngineTool(
        query_engine=query_engine_all,
        metadata=ToolMetadata(
            name="bi_ojk",
            description="Useful for retrieving answers from all documents.",
        )
    ),
]

In [124]:
system_prompt = """
You are a chatbot able to help answer questions about various types of regulations in Indonesia.

You MUST always answer WITH BAHASA INDONESIA.
You are NOT ALLOWED to hallucinate.
You MUST FOLLOW the answer from query engine.
You are ONLY ALLOWED to answer the question based on documents that you have been trained on.
You have access to a query tool to help you find relevant information in the regulations database.
Given the context information and not prior knowledge, answer the query using the query tool provided.
Use the previous chat history, or the context above, to interact and help the user.
----------------------------------------------
Query: {query_str}
Answer:
"""

In [125]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()
memory = ChatMemoryBuffer.from_defaults(
    token_limit=10000,
    chat_store=chat_store,
    chat_store_key="user1",
)

In [126]:
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [127]:
from utils.custom_agent import YourOpenAIAgent, CustomAgent, agent_with_memory
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.agent.legacy.react.base import ReActAgent

# agent = custom_agent(
#     llm=llm,
#     query_engine_tools=query_tools,
#     verbose=True,
# )

# agent = YourOpenAIAgent(
#     llm=llm,
#     tools=query_tools,
# )

# agent = agent_with_memory(
#     llm=llm,
#     tools=query_tools,
#     memory=memory,
#     system_prompt=system_prompt
# )

# agent = CustomAgent(
#     llm=llm,
#     query_engine_tools=query_tools,
#     verbose=True,
# )

agent = OpenAIAgent.from_tools(
    llm=llm,
    tools=query_tools,
    verbose=True,
    memory=memory,
    system_prompt=system_prompt,
    callback_manager=callback_manager,
)

# agent = ReActAgent.from_tools(
#     llm=llm,
#     memory=memory,
#     tools=query_tools,
#     verbose=True,
#     system_prompt=system_prompt,
#     callback_manager=callback_manager,
# )

In [128]:
# response_openai = agent_openai.chat(query_str)
# response_react = agent_react.chat(query_str)
response = agent.chat(message=query_str)

Added user message to memory: Berapa nilai SWDKLLJ Jeep?
=== Calling Function ===
Calling function: bi_ojk with args: {"input":"nilai SWDKLLJ Jeep"}


100%|██████████| 6/6 [00:01<00:00,  5.83it/s]


Got output: The SWDKLLJ for a Jeep according to the regulation varies depending on the engine capacity. For a Jeep with an engine capacity up to 1600 cc, the SWDKLLJ amount is Rp 140.000,00 (seratus empat puluh ribu rupiah).

**********
Trace: chat
    |_agent_step -> 6.049404 seconds
      |_llm -> 0.836425 seconds
      |_function_call -> 4.134056 seconds
      |_llm -> 1.071843 seconds
**********


In [129]:
response.source_nodes[0].metadata

{'file_name': 'ojk-klasifikasi_bapepam-36_pmk_010_2008-26022008-pmk_nomor_36pmk_010_tahun_2008_tentang_besar_santunan_dan_sumbangan_wajib_dana_kecelakaan_lalu_lintas_jalan_menas13_1389258036_pdf.pdf',
 'title': 'Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan',
 'sector': 'IKNB',
 'subsector': 'Asuransi',
 'regulation_type': 'Klasifikasi Bapepam',
 'regulation_number': '36/PMK.010/2008',
 'effective_date': '26 Februari 2008',
 'questions_this_excerpt_can_answer': '1. What is the effective date of Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan?\n- The effective date of the regulation is 26 Februari 2008.\n\n2. How much is the maximum amount of medical treatment that a victim of a traffic accident is entitled to under this regulation?\n- The maximum amount of medical treatment that a victim is entitled to is Rp 10.000.000,00 (ten mil

In [130]:
from llama_index.core.response.notebook_utils import display_response

display_response(response=response)

**`Final Response:`** Nilai SWDKLLJ untuk Jeep dengan kapasitas mesin hingga 1600 cc adalah Rp 140.000,00.

In [131]:
for node in response.source_nodes:
    print("**********")
    print("id: ", node.id_)
    print("content:\n", node.node.get_content())
    if node.score:
        print("reranking score: ", node.score)
    if node.node.metadata.get("retrieval_score"):
        print("retrieval score: ", node.node.metadata["retrieval_score"])
    print("**********")
    print("\n")

**********
id:  092263a4-30e0-4deb-8607-87a5cae21be5
content:
 c. Korban yang memerlukan perawatan dan pengobatan berhak pengobatan dokter paling besar Rp 10.000.000,00 (sepuluh juta rupiah). Pasal 3 Dalam hal korban meninggal dunia akibat kecelakaan alat angkutan lalu lintas jalan tidak mempunyai ahli waris, kepada pihak yang menyelenggarakan penguburan  diberikan  penggantian biaya penguburan sebesar Rp 2.000.000,00 (dua juta rupiah). Pasal 4 (1) SWDKLLJ dipungut dari para pengusaha/ pemilik alat angkutan lalu lintas jalan. (2) Besar SWDKLLJ sebagaimana dimaksud pada ayat (1) ditentukan sebagai berikut: a.  Sepeda motor di bawah 50 cc, mobil ambulance, mobil jenazah dan mobil pemadam kebakaran dibebaskan dari kewajiban membayar SWDKLLJ. b. Traktor, buldozer, forklift, mobil derek, excavator, crane dan sejenisnya sebesar Rp 20.000,00 (dua puluh ribu rupiah) C. s  Sepeda motor, sepeda kumbang dan scooter di atas 50 cc sampai 250 cc dan kendaraan bermotor roda tiga sebesar Rp 32.000,00 

In [132]:
chat_store_string = chat_store.json()
loaded_chat_store = SimpleChatStore.parse_raw(chat_store_string)